In [ ]:
#default_exp utils.box_np_ops

# Box operations from SECOND
> Here we used some box operations from the SECOND repo (https://github.com/traveller59/second.pytorch).


In [ ]:
#export
from pathlib import Path
import torch
import numba
import numpy as np

In [ ]:
#export
def second_box_encode(boxes,
                      anchors,
                      encode_angle_to_vector=False,
                      smooth_dim=False):
    """box encode for VoxelNet in lidar
    Args:
        boxes ([N, 7 + ?] Tensor): normal boxes: x, y, z, w, l, h, r, custom values
        anchors ([N, 7] Tensor): anchors
    """
    # need to convert boxes to z-center format
    box_ndim = anchors.shape[-1]
    cas, cgs = [], []
    if box_ndim > 7:
        xa, ya, za, wa, la, ha, ra, *cas = np.split(anchors, box_ndim, axis=1)
        xg, yg, zg, wg, lg, hg, rg, *cgs = np.split(boxes, box_ndim, axis=1)
    else:
        xa, ya, za, wa, la, ha, ra = np.split(anchors, box_ndim, axis=1)
        xg, yg, zg, wg, lg, hg, rg = np.split(boxes, box_ndim, axis=1)

    diagonal = np.sqrt(la**2 + wa**2)  # 4.3
    xt = (xg - xa) / diagonal
    yt = (yg - ya) / diagonal
    zt = (zg - za) / ha  # 1.6
    lt = np.log(lg / la)
    wt = np.log(wg / wa)
    ht = np.log(hg / ha)
    rt = rg - ra
    cts = [g - a for g, a in zip(cgs, cas)]
    if smooth_dim:
        lt = lg / la - 1
        wt = wg / wa - 1
        ht = hg / ha - 1
    else:
        lt = np.log(lg / la)
        wt = np.log(wg / wa)
        ht = np.log(hg / ha)
    if encode_angle_to_vector:
        rgx = np.cos(rg)
        rgy = np.sin(rg)
        rax = np.cos(ra)
        ray = np.sin(ra)
        rtx = rgx - rax
        rty = rgy - ray
        return np.concatenate([xt, yt, zt, wt, lt, ht, rtx, rty, *cts], axis=1)
    else:
        rt = rg - ra
        return np.concatenate([xt, yt, zt, wt, lt, ht, rt, *cts], axis=1)

In [ ]:
#export
def corners_nd(dims, origin=0.5):
    """generate relative box corners based on length per dim and
    origin point.

    Args:
        dims (float array, shape=[N, ndim]): array of length per dim
        origin (list or array or float): origin point relate to smallest point.
        dtype (output dtype, optional): Defaults to np.float32

    Returns:
        float array, shape=[N, 2 ** ndim, ndim]: returned corners.
        point layout example: (2d) x0y0, x0y1, x1y0, x1y1;
            (3d) x0y0z0, x0y0z1, x0y1z0, x0y1z1, x1y0z0, x1y0z1, x1y1z0, x1y1z1
            where x0 < x1, y0 < y1, z0 < z1
    """
    ndim = int(dims.shape[1])
    dtype = torch_to_np_dtype(dims.dtype)
    if isinstance(origin, float):
        origin = [origin] * ndim
    corners_norm = np.stack(
        np.unravel_index(np.arange(2**ndim), [2] * ndim), axis=1).astype(dtype)
    # now corners_norm has format: (2d) x0y0, x0y1, x1y0, x1y1
    # (3d) x0y0z0, x0y0z1, x0y1z0, x0y1z1, x1y0z0, x1y0z1, x1y1z0, x1y1z1
    # so need to convert to a format which is convenient to do other computing.
    # for 2d boxes, format is clockwise start from minimum point
    # for 3d boxes, please draw them by your hand.
    if ndim == 2:
        # generate clockwise box corners
        corners_norm = corners_norm[[0, 1, 3, 2]]
    elif ndim == 3:
        corners_norm = corners_norm[[0, 1, 3, 2, 4, 5, 7, 6]]
    corners_norm = corners_norm - np.array(origin, dtype=dtype)
    corners_norm = torch.from_numpy(corners_norm).type_as(dims)
    corners = dims.view(-1, 1, ndim) * corners_norm.view(1, 2**ndim, ndim)
    return corners


def corners_2d(dims, origin=0.5):
    """generate relative 2d box corners based on length per dim and
    origin point.

    Args:
        dims (float array, shape=[N, 2]): array of length per dim
        origin (list or array or float): origin point relate to smallest point.
        dtype (output dtype, optional): Defaults to np.float32

    Returns:
        float array, shape=[N, 4, 2]: returned corners.
        point layout: x0y0, x0y1, x1y1, x1y0
    """
    return corners_nd(dims, origin)

In [ ]:
#export
@numba.jit(nopython=True)
def iou_jit(boxes, query_boxes, eps=1.0):
    """calculate box iou. note that jit version runs 2x faster than cython in
    my machine!
    Parameters
    ----------
    boxes: (N, 4) ndarray of float
    query_boxes: (K, 4) ndarray of float
    Returns
    -------
    overlaps: (N, K) ndarray of overlap between boxes and query_boxes
    """
    N = boxes.shape[0]
    K = query_boxes.shape[0]
    overlaps = np.zeros((N, K), dtype=boxes.dtype)
    for k in range(K):
        box_area = ((query_boxes[k, 2] - query_boxes[k, 0] + eps) *
                    (query_boxes[k, 3] - query_boxes[k, 1] + eps))
        for n in range(N):
            iw = (min(boxes[n, 2], query_boxes[k, 2]) - max(
                boxes[n, 0], query_boxes[k, 0]) + eps)
            if iw > 0:
                ih = (min(boxes[n, 3], query_boxes[k, 3]) - max(
                    boxes[n, 1], query_boxes[k, 1]) + eps)
                if ih > 0:
                    ua = (
                        (boxes[n, 2] - boxes[n, 0] + eps) *
                        (boxes[n, 3] - boxes[n, 1] + eps) + box_area - iw * ih)
                    overlaps[n, k] = iw * ih / ua
    return overlaps

def get_iou(bb1, bb2):
    """
    Calculate the Intersection over Union (IoU) of two bounding boxes.

    Parameters
    ----------
    bb1 : dict
        Keys: {'x1', 'x2', 'y1', 'y2'}
        The (x1, y1) position is at the top left corner,
        the (x2, y2) position is at the bottom right corner
    bb2 : dict
        Keys: {'x1', 'x2', 'y1', 'y2'}
        The (x, y) position is at the top left corner,
        the (x2, y2) position is at the bottom right corner

    Returns
    -------
    float
        in [0, 1]
    """
    assert bb1['x1'] < bb1['x2']
    assert bb1['y1'] < bb1['y2']
    assert bb2['x1'] < bb2['x2']
    assert bb2['y1'] < bb2['y2']

    # determine the coordinates of the intersection rectangle
    x_left = max(bb1['x1'], bb2['x1'])
    y_top = max(bb1['y1'], bb2['y1'])
    x_right = min(bb1['x2'], bb2['x2'])
    y_bottom = min(bb1['y2'], bb2['y2'])

    if x_right < x_left or y_bottom < y_top:
        return 0.0

    # The intersection of two axis-aligned bounding boxes is always an
    # axis-aligned bounding box
    intersection_area = (x_right - x_left + 1) * (y_bottom - y_top + 1)

    # compute the area of both AABBs
    bb1_area = (bb1['x2'] - bb1['x1'] + 1) * (bb1['y2'] - bb1['y1'] + 1)
    bb2_area = (bb2['x2'] - bb2['x1'] + 1) * (bb2['y2'] - bb2['y1'] + 1)

    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = intersection_area / float(bb1_area + bb2_area - intersection_area)
    assert iou >= 0.0
    assert iou <= 1.0
    return iou



    # DEPRECATED
def calculate_iou_deprecated(self, pred_boxes: torch.Tensor, gt_boxes: torch.Tensor):
    """Calculates the iou between predicted boxes and ground truth boxes.

    :param pred_boxes: predicted boxes in form of Tensor(batch_size, nb_pred_boxes, 4) with the latter being
                        (x_min, y_min, x_max, y_max)
    :param gt_boxes: ground truth boxes in form of Tensor(batch_size, max_nb_gt_boxes, 4) with the latter
                        being (x_min, y_min, x_max, y_max)

    :returns: Tensor(batch_size, nb_pred_boxes, max_nb_gt_boxes_per_batch)
    """
    num_pred_boxes = pred_boxes.shape[1]
    num_gt_boxes = gt_boxes.shape[1]

    # both have shape Tensor(nb_batches, nb_pred_boxes, nb_gt_boxes, 4)
    pred_boxes = pred_boxes.unsqueeze(2).expand(-1, -1, num_gt_boxes, -1)
    gt_boxes = gt_boxes.unsqueeze(1).expand(-1, num_pred_boxes, -1, -1)

    # maximum calculations
    x_left = torch.max(pred_boxes[:,:,:,0], gt_boxes[:,:,:,0])
    y_top = torch.max(pred_boxes[:,:,:,1], gt_boxes[:,:,:,1])
    x_right = torch.max(pred_boxes[:,:,:,2], gt_boxes[:,:,:,2])
    y_bottom = torch.max(pred_boxes[:,:,:,3], gt_boxes[:,:,:,3])

    intersection_area = (x_right - x_left + 1) * (y_bottom - y_top + 1)
    #intersection_area = torch.where(x_right < x_left, intersection_area, 0.)
    #intersection_area = torch.where(y_bottom < y_top, intersection_area, 0.)
    print(intersection_area, intersection_area.shape)